In [1]:
import pandas as pd 
import numpy as np 
from pandas.api.types import is_string_dtype
from pandas.api.types import is_numeric_dtype
from matplotlib import pyplot as plot

In [3]:
dataDir="/home/colette_berg/YNP/AHQsd_2021/stacks/"
genotypes="AHQsd_forPhasing_filtered.012"

In [4]:
# read in the data -- genotypes, sites, and individuals and phase.

## there's one dataframe with T & N and another with the rest of the individuals.
NT_data = pd.read_csv(dataDir + "NT_forPhasing.012", sep="\t", index_col=False, header=None).iloc[:, 1:]
NT_indv = pd.read_csv(dataDir + "NT_forPhasing.012.indv", sep="\t", index_col=False, header=None)

data_genotypes = pd.read_csv(dataDir + genotypes, sep="\t", index_col=False, header=None).iloc[:, 1:]
data_indv = pd.read_csv(dataDir + genotypes + ".indv", sep="\t", index_col=False, header=None)
data_sites = pd.read_csv(dataDir + genotypes + ".pos", sep="\t", index_col=False, header=None)

# format the names of the sites
tmpDF = pd.DataFrame(columns=['scaffold','chr'])
tmpDF[['scaffold','chr']] = data_sites[0].str.split('_', expand=True)

data_sites['chr'] = tmpDF['chr']
data_sites['site'] = data_sites[0].astype(str) + ['_'] + data_sites[1].astype(str)

# paste NT data and AHQsd data together
AHQsd_and_NT = pd.concat([NT_data, data_genotypes])

all_indv = pd.concat([NT_indv, data_indv])

# add columns and row names 
AHQsd_and_NT.columns = data_sites['site']
AHQsd_and_NT.insert(0, 'indv', all_indv[0])

# transpose for filtering fun
genotypes_transposed = AHQsd_and_NT.set_index('indv').T

# replace -1 with NaN
genotypes_transposed = genotypes_transposed.replace(-1, np.NaN)

genotypes_transposed.reset_index(inplace=True)


In [5]:

# get rid of any sites where T and N are hets
# genotypes_transposed = genotypes_transposed[genotypes_transposed['AHQNT1.6_8'] != 1]
# genotypes_transposed = genotypes_transposed[genotypes_transposed['AHQT1'] != 1]

# make a T and an N dataframe
N_is_ref = genotypes_transposed[genotypes_transposed['AHQNT1.6_8'] == 0.0]
T_is_ref = genotypes_transposed[genotypes_transposed['AHQT1'] == 0.0]


## replace the genotypes for N
N_is_ref = N_is_ref.replace(0.0, "N")
N_is_ref = N_is_ref.replace(1.0, "NT")
N_is_ref = N_is_ref.replace(2.0, "T")

## replace the genotypes for T
T_is_ref = T_is_ref.replace(0.0, "T")
T_is_ref = T_is_ref.replace(1.0, "NT")
T_is_ref = T_is_ref.replace(2.0, "N")

phased_sites = pd.concat([N_is_ref, T_is_ref])

phased_sites.to_csv("AHQsd_phased.csv")
phased_sites.to_csv("AHQsd_phased_tab.csv", sep="\t")

In [6]:
phased_sites

indv,site,AHQNT1.6_8,AHQT1,AHQsd_1.02E,AHQsd_1.04A,AHQsd_1.04C,AHQsd_1.05D,AHQsd_1.07B,AHQsd_1.07G,AHQsd_1.09C,...,AHQsd_6.10G,AHQsd_6.10H,AHQsd_6.11B,AHQsd_6.11E,AHQsd_6.11H,AHQsd_6.12D,AHQsd_6.12E,AHQsd_6.12F,AHQsd_6.12G,AHQsd_6.12H
1,scaffold_1_24248,N,T,NT,NT,N,N,NT,N,NaN,...,NT,N,NT,N,N,T,NT,N,NT,N
2,scaffold_1_29968,N,T,NT,NT,N,N,NT,N,N,...,NT,N,NT,N,N,T,NT,N,NT,N
3,scaffold_1_29966,N,T,NT,NT,N,N,NT,N,N,...,NT,N,NT,N,N,T,NT,N,NT,N
4,scaffold_1_30111,N,T,N,NT,N,N,NT,NaN,N,...,NT,N,NT,N,N,T,NT,N,NT,N
5,scaffold_1_33601,N,T,NT,NT,N,N,NT,N,N,...,NT,N,NT,N,N,T,NT,N,NT,N
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9154,scaffold_3071_1693,N,T,NaN,NT,T,N,NT,NT,NaN,...,NT,NT,NT,NT,NT,NT,T,T,NT,NT
9155,scaffold_3071_1692,N,T,NaN,NT,T,N,NT,NT,NaN,...,NT,NT,NT,NT,NT,NT,T,T,NT,NT
9156,scaffold_3071_1691,N,T,NaN,NT,T,N,NT,NT,NaN,...,NT,NT,NT,NT,NT,NT,T,T,NT,NT
9157,scaffold_3605_2816,N,T,N,NT,NT,N,T,NT,NT,...,N,NaN,T,NT,NT,T,NT,N,N,N


In [40]:
tdf = phased_sites.set_index('site').T.reset_index()
# make a list of all the sites (these are the same as column names)
siteList = list(tdf)

refCount = []
hetCount = []
altCount = []

In [42]:
# make a dataframe containing chr numbers and site numbers
distortionDF = pd.DataFrame(columns=['genome','chr','site'])
distortionDF[['genome','chr','site']] = phased_sites['site'].str.split('_', expand=True)

# loop through and count all the ref, het, and alt genotypes at each site
for i in siteList:
    countT = (tdf[i]=="T").sum()
    refCount.append(countT)

for i in siteList:
    countHet = (tdf[i]=="NT").sum()
    hetCount.append(countHet)

for i in siteList:
    countN = (tdf[i]=="N").sum()
    altCount.append(countN)

In [43]:
# remove first column
altCount = altCount[1::]
hetCount = hetCount[1::]
refCount= refCount[1::]

# convert from int to numeric
altCount = pd.to_numeric(altCount)
hetCount = pd.to_numeric(hetCount)
refCount = pd.to_numeric(refCount)

# calculate the sum
total = altCount + hetCount + refCount

refRatio = np.divide(refCount, total)
hetRatio = np.divide(hetCount, total)
altRatio = np.divide(altCount, total)

/tmp/ipykernel_1310460/3934131492.py:14: RuntimeWarning: invalid value encountered in divide
  refRatio = np.divide(refCount, total)
/tmp/ipykernel_1310460/3934131492.py:15: RuntimeWarning: invalid value encountered in divide
  hetRatio = np.divide(hetCount, total)
/tmp/ipykernel_1310460/3934131492.py:16: RuntimeWarning: invalid value encountered in divide
  altRatio = np.divide(altCount, total)


In [8]:
distortionDF['refRatio'] = refRatio
distortionDF['hetRatio'] = hetRatio
distortionDF['altRatio'] = altRatio

In [44]:
c1 = distortionDF[distortionDF['chr'] == "1"]
c2 = distortionDF[distortionDF['chr'] == "2"]
c3 = distortionDF[distortionDF['chr'] == "3"]
c4 = distortionDF[distortionDF['chr'] == "4"]

c5 = distortionDF[distortionDF['chr'] == "5"]
c6 = distortionDF[distortionDF['chr'] == "6"]
c7 = distortionDF[distortionDF['chr'] == "7"]
c8 = distortionDF[distortionDF['chr'] == "8"]

c9 = distortionDF[distortionDF['chr'] == "9"]
c10 = distortionDF[distortionDF['chr'] == "10"]
c11 = distortionDF[distortionDF['chr'] == "11"]
c12 = distortionDF[distortionDF['chr'] == "12"]

c13 = distortionDF[distortionDF['chr'] == "13"]
c14 = distortionDF[distortionDF['chr'] == "14"]

In [46]:
plot.plot(pd.to_numeric(c1['site']), c1['refRatio'], 'ro')

KeyError: 'refRatio'